# Uncertainty Quantification for test cases

In [1]:
import os

import numpy as np
from scipy.stats import norm,uniform

import UQLib.sample.TMCMC as TMCMC
import test_cases.sinusoidal as sinusoidal

np.random.seed(1234)

## Inverse UQ for a sinusoidal function

In [2]:
modelpath = "/home/kevin/master_project/IUQ-Project/test_cases/sinusoidal.py"

# Generate data with Gaussian noise
x, y, y_err, true_params = sinusoidal.generate(3)

model_params = ["a","b","c","k1","k2","f1","f2"]

print("True parameter values:")
print(tuple(model_params), "=", true_params)

error_params = ["s"]
design_vars = ["x"]

model_errors = ["s" for n in range(x.shape[0])]

# Define uniform prior wrapper functions and samplers for the model parameters
uniform_func = lambda val: uniform(val)
uniform_sampler = lambda n_samples: np.random.uniform(-1,1,size=n_samples)

model_priors = [(uniform_func,uniform_sampler) for param in model_params]

# Define prior wrapper function and sampler for the error parameter
normal_func = lambda val: norm.pdf(val,0.0,1.0)
normal_sampler = lambda n_samples: np.random.normal(0.0,1.0,size=n_samples)

error_priors = [(normal_func,normal_sampler)]

# Construct problem dict
problem = {"setup":sinusoidal.setup,
           "measure":sinusoidal.read_output,
           "modelpath":modelpath,
           "model parameters":model_params,
           "error parameters":error_params,
           "design variables":design_vars,
           "input data":x,
           "output data":y,
           "data errors":y_err,
           "error mapping":model_errors,
           "model priors":model_priors,
           "error priors":error_priors
          }

likelihoods = TMCMC.sample(problem,3)

print(likelihoods)

True parameter values:
('a', 'b', 'c', 'k1', 'k2', 'f1', 'f2') = [-0.6169611   0.24421754 -0.12454452  0.57071717  0.55995162 -0.45481479
 -0.44707149]
Batch: batch_0, Run: run_0 running...
Batch: batch_0, Run: run_1 running...
Batch: batch_0, Run: run_2 running...
Batch: batch_1, Run: run_0 running...
Batch: batch_1, Run: run_1 running...
Batch: batch_1, Run: run_2 running...
Batch: batch_2, Run: run_0 running...
Batch: batch_2, Run: run_1 running...
Batch: batch_2, Run: run_2 running...
[7.82034915e-06 1.86272466e-02 6.39385514e-03]
